# Приложение для тех, кто любит читать.

**Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.**

### Цели исследования - сформулировать ценностное предложение для нового продукта.

## Подготовка данных и краткий обзор

In [1]:
#pip install sqlalchemy
#pip install pycopyg2-binary

In [2]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
    'pwd': 'my_pass', # пароль
    'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
    'port': 6432, # порт подключения
    'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
# какие ещё таблицы доступны в базе
display(pd.io.sql.read_sql('''

SELECT *
FROM pg_catalog.pg_tables
WHERE schemaname != 'pg_catalog' AND
      schemaname != 'information_schema';

''', con = engine))


# смотрим на типы столбцов в интересующих таблицах
display(pd.io.sql.read_sql('''
SELECT 
    table_name, 
    column_name, 
    data_type, 
    is_nullable
FROM INFORMATION_SCHEMA.COLUMNS 
WHERE table_name IN ('books', 'authors', 'publishers', 'ratings', 'reviews');
''', con = engine))

,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers,rowsecurity
0,public,orders,praktikum_admin,None,True,False,False,False
1,public,visits,praktikum_admin,None,True,False,False,False
2,public,advertisment_costs,praktikum_admin,None,True,False,False,False
3,public,authors,praktikum_admin,None,True,False,True,False
4,public,second,praktikum_student,None,False,False,False,False
5,public,second_b,praktikum_student,None,False,False,False,False
6,public,publishers,praktikum_admin,None,True,False,True,False
7,public,author,praktikum_student,None,True,False,False,False
8,public,reviews,praktikum_admin,None,True,False,True,False
9,public,ratings,praktikum_admin,None,True,False,True,False


,table_name,column_name,data_type,is_nullable
0,authors,author_id,integer,NO
1,authors,author,text,YES
2,publishers,publisher_id,integer,NO
3,publishers,publisher,text,YES
4,reviews,review_id,integer,NO
5,reviews,book_id,integer,YES
6,reviews,username,text,YES
7,reviews,text,text,YES
8,ratings,rating_id,integer,NO
9,ratings,book_id,integer,YES


## Задачи

### Посчитайте, сколько книг вышло после 1 января 2000 года;

In [4]:
query = '''SELECT COUNT(book_id)
        FROM books
        WHERE publication_date >= '2000-01-01' '''

pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,821


**С января 2000 года мы загрузили 821 книгу**

### Для каждой книги посчитайте количество обзоров и среднюю оценку

In [5]:
query = '''SELECT ra.book_id, books.title,
                  AVG(ra.rating) AS rating_mean,
                  COUNT(DISTINCT re.review_id) AS reviews_cnt
        FROM ratings AS ra
        FULL OUTER JOIN reviews AS re ON ra.book_id=re.book_id
        JOIN books ON ra.book_id=books.book_id
        GROUP BY ra.book_id, books.title '''

pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,title,rating_mean,reviews_cnt
0,1,'Salem's Lot,3.666667,2
1,2,1 000 Places to See Before You Die,2.500000,1
2,3,13 Little Blue Envelopes (Little Blue Envelope...,4.666667,3
3,4,1491: New Revelations of the Americas Before C...,4.500000,2
4,5,1776,4.000000,4
...,...,...,...,...
995,996,Wyrd Sisters (Discworld #6; Witches #2),3.666667,3
996,997,Xenocide (Ender's Saga #3),3.400000,3
997,998,Year of Wonders,3.200000,4
998,999,You Suck (A Love Story #2),4.500000,2


**На одну книгу можно получить до 7 обзоров.**

**Самый низкий рейтинг = 1,5. Максимальный рейтинг = 5.**

### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц 

In [6]:
query = '''SELECT pu.publisher_id, pu.publisher, COUNT(book_id)
        FROM publishers AS pu
        JOIN books ON pu.publisher_id=books.publisher_id
        WHERE books.num_pages > 50
        GROUP BY pu.publisher_id
        ORDER BY COUNT(book_id) DESC
        LIMIT 1'''

pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher,count
0,212,Penguin Books,42


**Лидирует издательство Penguin Books с количеством книг = 42**

### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками 

In [7]:
query = '''SELECT au.author_id, au.author, round(AVG(r.rating), 3) AS rating_mean, COUNT(r.rating) AS cnt_rating
        FROM authors AS au
        JOIN books ON au.author_id=books.author_id
        JOIN ratings AS r ON books.book_id=r.book_id
        GROUP BY au.author_id, au.author
        HAVING COUNT(r.rating) > 49
        ORDER BY cnt_rating DESC
        LIMIT 1'''

pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author,rating_mean,cnt_rating
0,236,J.K. Rowling/Mary GrandPré,4.288,312


**У автора J.K. Rowling/Mary GrandPré самая высокая средняя оценка книг среди коллег - 4,288.**

### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [8]:
query = '''SELECT AVG(review_cnt) AS review_mean_cnt
            FROM (SELECT COUNT(review_id) AS review_cnt
                  FROM reviews
                  WHERE username IN (SELECT username
                                    FROM ratings
                                    GROUP BY username
                                    HAVING COUNT(rating) > 48)
                  GROUP BY username) AS re '''

pd.io.sql.read_sql(sql=text(query), con = con)

,review_mean_cnt
0,24.0


**`24` - среднее количество обзоров от пользователей, которые поставили больше 48 оценок.**

## Выводы

* С января 2000 года мы загрузили `821 книгу`.

    
* На одну книгу можно получить `до 7 обзоров`.


* Самый низкий рейтинг = 1,5. Максимальный рейтинг = 5.

    
* Издательство, которое выпустило наибольшее число книг - `Penguin Books`.

    
* У автора `J.K. Rowling/Mary GrandPré` самая высокая средняя оценка книг среди коллег - `4,288`.

    
* `24` - среднее количество обзоров от пользователей, которые поставили больше 48 оценок.